In [28]:
import os
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
import xgboost as xgb

In [17]:
# LLM Dataset
data = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
data = data[["text", "generated"]].copy()

In [3]:
temp1 = pd.read_csv('/kaggle/input/daigt-external-train-dataset/train_external_drcat_02.csv')
temp1 = temp1[["text", "label"]].copy()
temp1.rename(columns={"label": "generated"}, inplace=True)
temp1.dropna(inplace=True)

/tmp/ipykernel_216/3124174556.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  temp1 = pd.read_csv('/kaggle/input/daigt-external-train-dataset/train_external_drcat_02.csv')


In [5]:
t1 = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv')
t2 = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_02.csv')
t3 = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_03.csv')
t4 = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv')

temp2 = pd.concat([t1, t2, t3, t4], ignore_index=True)
temp2 = temp2[["text", "label"]].copy()
temp2.rename(columns={"label": "generated"}, inplace=True)
temp2.dropna(inplace=True)

In [6]:
t1 = pd.read_csv('/kaggle/input/daigt-v3-train-dataset/train_v3_drcat_01.csv')
t2 = pd.read_csv('/kaggle/input/daigt-v3-train-dataset/train_v3_drcat_02.csv')

temp3 = pd.concat([t1, t2], ignore_index=True)
temp3.rename(columns={"label": "generated"}, inplace=True)
temp3.dropna(inplace=True)

In [18]:
data = pd.concat([data, temp1, temp2, temp3], ignore_index=True)

In [19]:
# data = data.sample(frac=0.1, random_state=3).reset_index(drop=True)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(data["text"], data["generated"], test_size=0.01)

In [21]:
vectorizer = TfidfVectorizer(max_features=None)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [34]:
model = xgb.XGBClassifier(objective='binary:logistic', eval_metric='auc')
model.fit(X_train_vectorized, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [35]:
accuracy = model.score(X_test_vectorized, y_test)
print(f"Accuracy: {accuracy}")
prob = model.predict_proba(X_test_vectorized)
prob = prob[:,1]
roc_auc = roc_auc_score(y_test, prob)
print("ROC AUC Score:", roc_auc)

Accuracy: 0.9786324786324786
ROC AUC Score: 0.9993899184015862


In [36]:
input_data = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
input_data = input_data[["id", "text"]].copy()

In [37]:
model_input = vectorizer.transform(input_data["text"])
pred = model.predict_proba(model_input)
pred = pred[:, 1]

In [38]:
output_df = pd.DataFrame({
    "id": input_data["id"],
    "generated": pred
})
output_df.to_csv('/kaggle/working/submission.csv', index=False)